# Experimentos Finales Modelos no RN

In [38]:
import pandas as pd
import numpy as np

import missingno as msno
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split 
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import datetime
from dateutil.parser import parse
from sklearn.tree import  DecisionTreeClassifier
import plotly.graph_objects as go
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
import xgboost as xgb
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
import ml_metrics
import pickle
from sklearn.metrics import accuracy_score
import joblib
import json


In [61]:
Data=pd.read_csv("cleaned_05June2022.csv").drop(columns=["Unnamed: 0"]).fillna("")
Data.head()

c:\Users\Jose Luis\anaconda3\envs\Programacion_Cientifica\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (36) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Duracion_Campaña,B,C,lec_B-B,lec_E-E,lec_D-E,lec_C-D,lec_A-A,num_comunicaciones,Length,...,Renta,Recibe_sueldo_en_cuenta,Segmento_consumidor,Meses_antiguedad,Comuna,Ciudad,Estado_civil,Principalidad,Profesion,Target
0,1.0,4.0,5.0,5.0,0.0,0.0,0.0,0.0,22.0,371,...,R1,0,A,Mayor a 10 años,331.0,13.0,D,B,P164,
1,1.0,6.0,6.0,8.0,0.0,1.0,1.0,0.0,15.0,371,...,R4,0,A,Mayor a 10 años,91.0,13.0,D,B,P85,E-E B-B D-E
2,1.0,2.0,3.0,1.0,0.0,0.0,1.0,0.0,26.0,371,...,R5,0,A,Mayor a 10 años,34.0,5.0,B,C,P143,D-E A-A
3,1.2,7.0,3.0,5.0,0.0,0.0,2.0,2.0,34.0,371,...,R9,0,A,Mayor a 10 años,331.0,13.0,B,C,P1,E-E
4,1.0,4.0,1.0,1.0,0.0,0.0,0.0,0.0,24.0,371,...,R2,0,B,Mayor a 10 años,37.0,5.0,D,F,P210,D-E


In [40]:
#k=0
#my_dic={}
#for i in Data["Target"].unique():
#    my_dic[k]=i
#    k=k+1

In [41]:
#tf = open("myDictionary.json", "w")
#json.dump(my_dic,tf)
#tf.close()

In [62]:
tf = open("myDictionary.json", "r")
dic_target = json.load(tf)
print(dic_target)

{'0': '', '1': 'E-E B-B D-E', '2': 'D-E A-A', '3': 'E-E', '4': 'D-E', '5': 'D-E B-B', '6': 'D-E B-B A-A', '7': 'B-B E-E', '8': 'D-E E-E B-B', '9': 'B-B D-E', '10': 'A-A B-B', '11': 'C-D B-B D-E', '12': 'E-E B-B', '13': 'B-B', '14': 'C-D E-E', '15': 'E-E C-D', '16': 'C-D B-B', '17': 'C-D', '18': 'D-E E-E', '19': 'C-D D-E B-B', '20': 'E-E D-E', '21': 'C-D E-E D-E', '22': 'B-B D-E E-E', '23': 'E-E D-E B-B', '24': 'C-D D-E', '25': 'B-B E-E D-E', '26': 'B-B A-A', '27': 'A-A', '28': 'D-E A-A B-B', '29': 'A-A D-E', '30': 'D-E C-D B-B', '31': 'D-E C-D', '32': 'D-E B-B E-E', '33': 'B-B C-D', '34': 'D-E B-B C-D'}


In [63]:
inv_map = {v: k for k, v in dic_target.items()}

In [64]:
# datos númericos
numerical=list(Data.select_dtypes(exclude=["category","object"]).columns)
numerical.remove("Sexo")
numerical.remove('Recibe_sueldo_en_cuenta')


# datos categorical
categorical=list(Data.select_dtypes(include=["category","object"]).columns)+['Sexo','Recibe_sueldo_en_cuenta']
categorical.remove("Target")

In [65]:
for i in categorical:
    Data[i]=Data[i].astype(str)

In [66]:
# Select 1 - Only Gini, corte visual
select_variables_1=['E-F',
 'D-F',
 'C-C',
 'A-K',
 'A-G',
 'num_comunicaciones',
 'Recency',
 'Positivo',
 'B',
 'Monto',
 'Periodicity',
 'lec_B-B',
 'F-I',
 'lec_A-A',
 'Sin Info',
 'Duracion_Campaña',
]

# Train, Test Split

In [67]:
X=Data.drop(columns=["Target"]).copy()
X.head()

,Duracion_Campaña,B,C,lec_B-B,lec_E-E,lec_D-E,lec_C-D,lec_A-A,num_comunicaciones,Length,...,Sexo,Renta,Recibe_sueldo_en_cuenta,Segmento_consumidor,Meses_antiguedad,Comuna,Ciudad,Estado_civil,Principalidad,Profesion
0,1.0,4.0,5.0,5.0,0.0,0.0,0.0,0.0,22.0,371,...,1,R1,0,A,Mayor a 10 años,331.0,13.0,D,B,P164
1,1.0,6.0,6.0,8.0,0.0,1.0,1.0,0.0,15.0,371,...,1,R4,0,A,Mayor a 10 años,91.0,13.0,D,B,P85
2,1.0,2.0,3.0,1.0,0.0,0.0,1.0,0.0,26.0,371,...,1,R5,0,A,Mayor a 10 años,34.0,5.0,B,C,P143
3,1.2,7.0,3.0,5.0,0.0,0.0,2.0,2.0,34.0,371,...,1,R9,0,A,Mayor a 10 años,331.0,13.0,B,C,P1
4,1.0,4.0,1.0,1.0,0.0,0.0,0.0,0.0,24.0,371,...,0,R2,0,B,Mayor a 10 años,37.0,5.0,D,F,P210


In [68]:
Y=Data["Target"]
Y.head()

0               
1    E-E B-B D-E
2        D-E A-A
3            E-E
4            D-E
Name: Target, dtype: object

In [69]:
Y.replace(inv_map,inplace=True)
Y

0         0
1         1
2         2
3         3
4         4
         ..
65025    16
65026    17
65027    25
65028     0
65029     5
Name: Target, Length: 65030, dtype: object

In [70]:
# Generación de conjuntos de train y test
Xtrain, Xtest, Ytrain, Ytest = train_test_split(
    X, Y, test_size=0.33, 
    shuffle=True,
    
) 

In [71]:
# Sobremuestreo: Aumentar numero de datos de la clase minoritaria
# ROS(Duplica clases):
ros=RandomOverSampler(random_state=0) # Random_state=0

In [72]:
# ROS:
Xtrain, Ytrain=ros.fit_resample(Xtrain,Ytrain)

In [73]:
Xtrain.shape

(284270, 39)

In [74]:
numerical1=list(filter(lambda x:x  in numerical,select_variables_1))
categorical1=list(filter(lambda x:x  in categorical,select_variables_1))

In [75]:
# Preprocesamiento de datos
preprocessing_transformer1 = ColumnTransformer(
    transformers=[('OneHotEncoder', OneHotEncoder(handle_unknown="ignore"),categorical1 ),
        ('MinMax', MinMaxScaler(),numerical1),
    ])

In [76]:
pipe1 = Pipeline(
    [("preprocesamiento", preprocessing_transformer1)]
)

In [77]:
# Preprocesamiento
pipe1.fit(Xtrain)
Xtrain1=pipe1.transform(Xtrain)
Xtest1=pipe1.transform(Xtest)


In [78]:
model= xgb.XGBClassifier(seed=1,  eval_metric='mlogloss',
                               learning_rate= 0.05,n_estimators= 500,
                               #tree_method='gpu_hist', gpu_id=0
                               )

In [68]:
# Load
model_xgb1 = xgb.XGBClassifier()
model_xgb1.load_model("model_xgb1.bin")

In [80]:
# Fit
model.fit(Xtrain1, Ytrain)

c:\Users\Jose Luis\anaconda3\envs\Programacion_Cientifica\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              eval_metric='mlogloss', gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.05, max_delta_step=0,
              max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=500, n_jobs=8,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=1, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              seed=1, subsample=1, tree_method='exact', validate_parameters=1, ...)

In [101]:
# Generación de predicciones
Ypred =pd.DataFrame(model.predict(Xtest1))

In [89]:
accuracy_score(Ypred, Ytest)


0.3662628145386766

In [103]:
Ypred.replace(dic_target,inplace=True)
Ytest.replace(dic_target,inplace=True)

In [118]:
Ypred_map=Ypred[0].apply( lambda x: x.split(" "))
Ytest_map=Ytest.apply( lambda x: x.split(" "))
ml_metrics.mapk(Ytest_map, Ypred_map, 5)

0.7190923682302993

In [119]:
# save to bin
model.save_model("./model_xgb1.bin")


In [120]:
# Guardar pipeline:
pickle.dump(pipe1, open('./pipe_xgb1.pkl', "wb"))